In [56]:
import re
import requests
from bs4 import BeautifulSoup
import shutil 
import os

In [57]:
num_page= int(input("Insert total number of pages(with 50 images in each page): "))
num_page+=1
url_start= 'https://gallica.bnf.fr/services/engine/search/sru?operation=searchRetrieve&version=1.2&startRecord='
url_middle='&maximumRecords=50&page='
search_term= str(input("Insert term to search images for: "))
url_end= f'&query=(gallica all "{search_term}")'
filter_end= '&filter=dc.type all "image"'

final_urls=[]

start_rec=0
for search in range(1,num_page):
    final_search=str(url_start+str(start_rec)+url_middle+str(search)+url_end+filter_end)
    start_rec+=50
    reqs = requests.get(final_search)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    prefix= 'https://gallica.bnf.fr/ark:'
    txt= '(?:(?:https?):\/\/)?[\w/\-?=%.]+\.highres'
    for url in re.findall(txt,str(soup)):
        suffix= url.replace(".","/f1.")
        all_together= prefix+suffix
        final_urls.append(all_together)

Insert total number of pages(with 50 images in each page): 10
Insert term to search images for: Mai 1968


In [ ]:
# stopped at Retrieve&version=1.2&startRecord=25550&maximumRecords=50&page=511&query= Mai 1968 Le Front populaire

In [59]:
search_term=search_term.replace(" ","_")

In [58]:
len(list(set(final_urls)))

380

In [60]:
final_urls=list(set(final_urls))

In [62]:
os.mkdir(os.path.join(os.getcwd(),"images_from_the_past",'FRANCE',search_term))

In [63]:
import pandas as pd
final_df= pd.DataFrame()
col_name= 'bnf_gallica_'+search_term
final_df[col_name]= final_urls
csv_name= col_name+".csv"

In [64]:
final_df.to_csv(os.path.join(os.getcwd(),"images_from_the_past",'FRANCE',csv_name),index=False)

In [65]:
# final_urls

In [66]:
# url= 'https://gallica.bnf.fr/ark:/12148/btv1b9018124t/f1.highres'

In [68]:
for url in final_urls:
    ## Set up the image URL and filename
    image_url = url
    filename = image_url.split("/")[-2]+".jpg"
    filename=filename.replace("/","_")
    filename= os.path.join(os.getcwd(),"images_from_the_past",'FRANCE',search_term,filename)
    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_url, stream = True)

    # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True

        # Open a local file with wb ( write binary ) permission.
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)

    #     print('Image sucessfully Downloaded: ',filename)
    else:
        print('Image Couldn\'t be retreived')

In [180]:
# import wget

# initial = 'https://gallica.bnf.fr/ark:/12148/btv1b9027504d.r=manifestations?rk=171674;4'

# text_split = initial.split('.')
# initial_no_end = initial.replace(text_split[-1],'')
# old = '.'
# new = '/f'
# maxreplace = 1

# result = new.join(initial_no_end.rsplit(old, maxreplace))

# result

# fraum = 1 # 1st page to download
# tou = 5 # Last page to download (overshoot is not an issue, will just add blank files)
# part1 = result  # exemple: couper la zone après f (folio ?) '4.highres' 'https://gallica.bnf.fr/ark:/12148/bpt6k408619m/f4.highres'

# while fraum <= tou:
#     no = str(fraum)
#     part2 = '.highres'
#     noext = no + ".jpg"
#     fraum = fraum + 1
#     image_url = part1 + no + part2
#     print('\r\npage ', fraum, '/', tou)
#     local_image_filename = wget.download(image_url, out=noext)